In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.decomposition import PCA

import sys
sys.path.append('..')

import neural_manifolds as nm

%load_ext autoreload
%autoreload 2

# Example: artifical data

In this notebook we show the general workflow on how to create artificial data. We will use the `artificial_data` submodule to create the data. The module is located in `neural_manifolds/artificial_data/*`, you can find all the functions there.

## 1 Latent model
Here, we construct spikes for $N$ neurons from time-dependent rates $\nu_i(t)$ that are linear combinations from a latent model with $z(t)$.

Specifically, we choose 
$$
   z_1(t) = sin(\pi \omega t)\\
   z_2(t) = cos(\pi \omega t )
$$

In [ ]:

latent_model = nm.test_data.LatentModel("sin_cos", omega=0.2)

# Define time vector
t = np.arange(0,5,0.001)
latent_state = latent_model(t)
print(latent_model)


In [ ]:
plt.plot(latent_state[0,:],latent_state[1,:]) 
plt.title("Latent space")

## 2. Firing rates

Then, we generate neural firing rates as linear combinations of the latent variables

$$
  \nu_i (t) = \sum_j c_{ij} \cdot z_j(t) + \mathrm{offset}
$$

In [ ]:
from neural_manifolds.test_data import firing_rates
# Number of neurons
N = 32
rates = firing_rates(latent_model, t, N)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 3))
ax.plot(
    rates.T, alpha=0.8
)
ax.set_title("Rates")
ax.set_xlabel("Time [s]")
plt.show()

# 3. Spikes
To get a bit more realistic, let's generate actual spikes as an inhomogeneous poisson process.

In [ ]:
spikes = nm.test_data.inhomogenous_poisson_spikes(rates, 0.001)

In [ ]:
ids = np.unique(spikes[:,0])

fig, ax = plt.subplots(1,1,figsize=(12, 5))
for id in ids:
    mask = spikes[:,0] == id
    spike_times = spikes[mask,1]
    ax.plot(spike_times, np.ones_like(spike_times)*id, '|', color='k', markersize=10)

ax.set_xlabel('Time [s]')
ax.set_ylabel("Neuron Id")
ax.set_title("Generated Spikes")
ax.set_xlim(0,t.max())
plt.show()

## 4. Signals

We transform the "real" spikes to signals, by creating a histogram of the spikes over time for each neuron.


In [ ]:
from scipy.ndimage import gaussian_filter1d
time_bin_width = 0.03 # in seconds
counts, _, time = np.histogram2d(
    spikes[:,0], # ids
    spikes[:,1], # times
    bins=[
        np.arange(N+1), # neurons
        np.arange(0,t.max(),time_bin_width) #times
        ]
)

sd_t = 0.05 #s
sqcounts = np.sqrt(counts)
sd=sd_t/time_bin_width
signal = gaussian_filter1d(sqcounts, sigma=sd, axis=1)



In [ ]:
fig, axes = plt.subplots(2,1,figsize=(12, 5))
axes[0].set_title("Bin counts")
axes[0].imshow(counts, aspect='auto', cmap='gray_r')

axes[1].set_title("Signals")
axes[1].plot(time[:-1], signal.T)
axes[1].set_xlim(0,time[:-1].max())
plt.tight_layout()
